To Do:

I need to create a Syft equivalent of nn.Module that lets you define a model if you initilize it with all the layers it will use, and define its forward pass using the activation functions.

SyMPC seems to make a Module equivalent but for each type of layer- i.e. Conv2d is implemented as a Module, and (I think) their MPCTensor tracks all its gradients.

Perhaps this will come down to modifying the loss function to take the DP Tensor as input, but I think if we use `publish` in order to do the conversion from DP Tensor -> array/tensor, we wouldn't have to do this modification


In [1]:
import syft

/opt/anaconda3/envs/Hagrid/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
from syft import nn

In [4]:
import syft.core.tensor.nn.functional as F

In [61]:
from typing import Union
from typing import Sequence
from typing import Tuple, Optional
from syft import PhiTensor

class Conv2d(torch.nn.Module):
    
    def __init__(self, in_channels:int, out_channels:int, kernel_size: Union[int, Sequence[int]], padding:int):
        super(Conv2d, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.func = torch.nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=self.kernel_size, padding=self.padding)
        
    def forward(self, x: PhiTensor):
        return nn.Conv2d(
            image=x, 
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            padding=self.padding
        )
    
    def parameters(self):
        return self.func.parameters()
    
    
class BatchNorm2d(torch.nn.Module):
    
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True):
        super(BatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.func = torch.nn.BatchNorm2d(num_features=self.num_features, 
            eps=self.eps, 
            momentum=self.momentum, 
            affine=self.affine
        )
        
    def forward(self, x: PhiTensor):
        return nn.BatchNorm2d(
            image=x, 
            num_features=self.num_features, 
            eps=self.eps, 
            momentum=self.momentum, 
            affine=self.affine
        )
    
    def parameters(self):
        return self.func.parameters()

class MaxPool2d(torch.nn.Module):
    
    def __init__(self, kernel_size: Union[int, Tuple[int, int]],
                 stride: Optional[Union[int, Tuple[int, int]]] = None,
                 padding: Union[int, Tuple[int, int]] = 0,
                 dilation: int = 1,
                 return_indices: bool = False,
                 ceil_mode: bool = False,
                ):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.return_indices = return_indices
        self.ceil_mode = ceil_mode
        self.func = torch.nn.MaxPool2d(kernel_size=self.kernel_size, 
            stride=self.stride, 
            padding=self.padding, 
            return_indices=self.return_indices, 
            ceil_mode=self.ceil_mode)
        
    def forward(self, x: PhiTensor):
        return nn.MaxPool2d(
            image=x, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            padding=self.padding, 
            return_indices=self.return_indices, 
            ceil_mode=self.ceil_mode
        )
    
    def parameters(self):
        return self.func.parameters()
    
    
class AvgPool2d(torch.nn.Module):
    def __init__(
        self, 
        kernel_size: Union[int, Tuple[int, int]], 
        stride: Optional[Union[int, Tuple[int, int]]] = None, 
        padding: Union[int, Tuple[int, int]] = 0
    ):
        super(AvgPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.func = torch.nn.AvgPool2d(kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
        
    def forward(self, x: PhiTensor):
        return nn.AvgPool2d(image=x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
    
    def parameters(self):
        return self.func.parameters()

    
class Linear(torch.nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.bias = bias
        self.func = torch.nn.Linear(in_features=self.in_features, out_features=self.out_features, bias=self.bias)
    
    def forward(self, x: PhiTensor):
        return nn.Linear(image=x, in_features=self.in_features, out_features=self.out_features, bias=self.bias)
    
    def parameters(self):
        return self.func.parameters()

In [62]:
class ConvNet(torch.nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = BatchNorm2d(32)
        self.bn2 = BatchNorm2d(64)
        self.bn3 = BatchNorm2d(128)
        self.bn4 = BatchNorm2d(256)
        self.bn5 = BatchNorm2d(512)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.avg = AvgPool2d(3)
        self.fc = Linear(512 * 1 * 1, 2)
        
    def forward(self, x: PhiTensor):
        # First layer of CNN - running 1 at a time to debug and see if any individual componenet is failing
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = F.leaky_relu(x)
#         x = self.pool(x)
        
        # Subsequent layers
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [63]:
cnn_model = ConvNet()

In [64]:
cnn_model

ConvNet(
  (conv1): Conv2d(
    (func): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  )
  (conv2): Conv2d(
    (func): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  )
  (conv3): Conv2d(
    (func): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  )
  (conv4): Conv2d(
    (func): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  )
  (conv5): Conv2d(
    (func): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  )
  (bn1): BatchNorm2d(
    (func): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn2): BatchNorm2d(
    (func): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn3): BatchNorm2d(
    (func): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn4): BatchNorm2d(
    (func): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn5):

In [65]:
from syft import PhiTensor
import numpy as np

N = 10
C_in = 3
H_in = 50
W_in = 50

input_shape = (N, C_in, H_in, W_in)
x = PhiTensor(child=np.random.randint(low=0, high=255, size=input_shape),
              data_subjects=np.zeros(input_shape),
              min_vals=0,
              max_vals=255
             )

In [66]:
prediction = cnn_model(x)

Alright so the results are terrible, but since it's random information I guess that's fine...

In [67]:
prediction.min_vals

<lazyrepeatarray data: [[0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]
 [0.02670823 0.01745377]] -> shape: (10, 2)>

In [68]:
prediction.max_vals

<lazyrepeatarray data: [[-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]
 [-6.609332  -2.1688619]] -> shape: (10, 2)>

In [69]:
prediction.child.decode()

array([[ -87105.8359375 ,  -29459.04882812],
       [ -92849.15625   ,  -28882.64648438],
       [ -92739.2421875 ,  -27127.921875  ],
       [ -96031.2578125 ,  -25348.609375  ],
       [ -83334.6171875 ,  -28440.17773438],
       [ -79925.5859375 ,  -25812.9375    ],
       [-103203.09375   ,  -28766.57226562],
       [ -88098.015625  ,  -28495.328125  ],
       [ -94136.6328125 ,  -24086.13671875],
       [ -92708.3203125 ,  -26001.5078125 ]])

In [72]:
epochs = 10
classes = 2
batch_size = 128
alpha = 0.002
device = 'cpu'

model = ConvNet().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=alpha)

In [87]:
def create_phi_tensor():
    return PhiTensor(
        child=np.random.randint(0, 255, (50, 50, 3)),
        data_subjects=np.ones((50, 50, 3)) * np.random.choice([0, 1]),
        min_vals=0,
        max_vals=255
    )

In [88]:
loader_train = [(create_phi_tensor(), torch.Tensor(np.random.choice([0, 1]))) for i in range(10)]

In [79]:
from tqdm import tqdm

In [91]:
from syft.core.adp.data_subject_ledger import DataSubjectLedger
from syft.core.adp.ledger_store import DictLedgerStore
from typing import Any


ledger_store = DictLedgerStore()
user_key = b"1231"
ledger = DataSubjectLedger.get_or_create(store=ledger_store, user_key=user_key)

def get_budget_for_user(*args: Any, **kwargs: Any) -> float:
    return 999999

def deduct_epsilon_for_user(*args: Any, **kwargs: Any) -> bool:
    return True

Creating new Ledger


In [95]:
total_step = len(loader_train)
for epoch in range(epochs):
    for i, (images, labels) in tqdm(enumerate(loader_train)):        
        # Forward pass
        outputs = model(images)
        published_output = outputs.publish(
            get_budget_for_user=get_budget_for_user, 
            deduct_epsilon_for_user=deduct_epsilon_for_user, 
            ledger=ledger, 
            sigma=10
        )
        loss = criterion(torch.Tensor(published_output.decode()), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

0it [00:00, ?it/s]

PUBLISHING TO GAMMA:
FixedPrecisionTensor(child=[[1639069184 1318499456]])



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 59.20it/s]
0it [00:00, ?it/s]

Epsilon spend  [706263.4037034 706263.4037034]
Highest possible spend  706263.403703397
Attemping to spend epsilon: 706263.403703397. Try: 0
got user budget 999999.0 epsilon_spent 706263.403703397
We have filtered all the input tensors. Now to compute the result:
Filtered inputs  [array([[1639069184, 1318499456]])]
original output (before noise: [[[1639069184 1318499456]]]
noise:  [[[20.32182046  7.32174347]]]
Noise after FPT [[[1331810  479837]]]
got output <class 'numpy.ndarray'> int64
Final FPT Values FixedPrecisionTensor(child=[1640400994 1318979293])


RuntimeError: size mismatch (got input: [2], target: [1])